# Project 3: NYC Car Collisons




The Motor Vehicle Collisions crash table contains details on the crash event. Each row represents a crash event. The Motor Vehicle Collisions data tables contain information from all police reported motor vehicle collisions in NYC. The police report (MV104-AN) is required to be filled out for collisions where someone is injured or killed, or where there is at least $1000 worth of damage 

https://data.cityofnewyork.us/Public-Safety/Motor-Vehicle-Collisions-Crashes/h9gi-nx95

Date Range: 8/12/2020 to 8/12/2023

https://data.cityofnewyork.us/Public-Safety/Motor-Vehicle-Collisions-Crashes/h9gi-nx95/explore/query/SELECT%0A%20%20%60crash_date%60%2C%0A%20%20%60crash_time%60%2C%0A%20%20%60borough%60%2C%0A%20%20%60zip_code%60%2C%0A%20%20%60latitude%60%2C%0A%20%20%60longitude%60%2C%0A%20%20%60location%60%2C%0A%20%20%60on_street_name%60%2C%0A%20%20%60off_street_name%60%2C%0A%20%20%60cross_street_name%60%2C%0A%20%20%60number_of_persons_injured%60%2C%0A%20%20%60number_of_persons_killed%60%2C%0A%20%20%60number_of_pedestrians_injured%60%2C%0A%20%20%60number_of_pedestrians_killed%60%2C%0A%20%20%60number_of_cyclist_injured%60%2C%0A%20%20%60number_of_cyclist_killed%60%2C%0A%20%20%60number_of_motorist_injured%60%2C%0A%20%20%60number_of_motorist_killed%60%2C%0A%20%20%60contributing_factor_vehicle_1%60%2C%0A%20%20%60contributing_factor_vehicle_2%60%2C%0A%20%20%60contributing_factor_vehicle_3%60%2C%0A%20%20%60contributing_factor_vehicle_4%60%2C%0A%20%20%60contributing_factor_vehicle_5%60%2C%0A%20%20%60collision_id%60%2C%0A%20%20%60vehicle_type_code1%60%2C%0A%20%20%60vehicle_type_code2%60%2C%0A%20%20%60vehicle_type_code_3%60%2C%0A%20%20%60vehicle_type_code_4%60%2C%0A%20%20%60vehicle_type_code_5%60%0AWHERE%0A%20%20%60crash_date%60%0A%20%20%20%20BETWEEN%20%222021-08-12T12%3A48%3A16%22%20%3A%3A%20floating_timestamp%0A%20%20%20%20AND%20%222023-08-12T12%3A48%3A16%22%20%3A%3A%20floating_timestamp/page/filter




In [ ]:
#Three libraries to bring at the onset

import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
matplotlib.style.use('ggplot')
%matplotlib inline
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go

In [ ]:

df = pd.read_csv(r'C:\Users\charl\OneDrive\Documents\Training\LaGuardia\Python\Motor_Vehicle_Collisions_Crashes.csv')

In [ ]:
df.head(5)

In [ ]:
df.columns

I noticed Zip Code in the data set. Thus, I searched and found a file that maps the city zip codes to the neighborhood names. The data also contained the population & density size for each zip code. I'd use this info. later on.

https://data.beta.nyc/en/dataset/pediacities-nyc-neighborhoods/resource/7caac650-d082-4aea-9f9b-3681d568e8a5  

In [ ]:
df_zip = pd.read_csv(r'C:\Users\charl\OneDrive\Documents\Training\LaGuardia\Python\nyc_zip_neighborhoods_pop.csv')

In [ ]:
df.describe()

In [ ]:
df.dtypes

In [ ]:
# I wanted to see how many records are in the data set.
df.shape[0]

In [ ]:
# I explored unique values in some of the columns that intrigued my interest.

df['VEHICLE TYPE CODE 1'].unique()                       

I wanted to create a column that adds up the items across multiple columns. The following codes helped me do exactly that. I counted the number of vehicles involved in each incident.

In [ ]:
df['TOTAL VEHICLES INVOLVED'] = df[['VEHICLE TYPE CODE 1', 'VEHICLE TYPE CODE 2', 'VEHICLE TYPE CODE 3','VEHICLE TYPE CODE 4', 'VEHICLE TYPE CODE 5']].count(axis=1) 

In [ ]:
df.head(5)

Again, I wanted to create a column that adds up the items across multiple columns. I add up the number of factors that caused each incident.

In [ ]:
df['TOTAL CONTRIBUTING FACTORS'] = df[['CONTRIBUTING FACTOR VEHICLE 1', 'CONTRIBUTING FACTOR VEHICLE 2', 'CONTRIBUTING FACTOR VEHICLE 3','CONTRIBUTING FACTOR VEHICLE 4', 'CONTRIBUTING FACTOR VEHICLE 5']].count(axis=1) 

In [ ]:
df.head(5)

I also wanted to create a column that extracted the year from CRASH DATE, whose data type was a string. Thus. I extracted the last four characters from each cell.

In [ ]:
df['YEAR'] = df['CRASH DATE'].str[-4:]

In [ ]:
df.head(5)

I needed to join the zip code to the name, population size and density to each neighborhood in the file, nyc_zip_neighborhoods_pop.csv. 

In [ ]:
df2 = df.set_index('ZIP CODE').join(df_zip.set_index('ZIP CODE'), how='inner')


In [ ]:
df2.head(5)

In [ ]:
df2.columns

I aggregated first by Borough.

In [ ]:
df3 = df2.groupby(['BOROUGH'])\
                      .agg(
                        Incidents = ('COLLISION_ID', 'count'),
                        Neighborhoods = ('NEIGHBORHOOD', 'nunique'),
                        Injured = ('NUMBER OF PERSONS INJURED', 'sum'),
                        Killed = ('NUMBER OF PERSONS KILLED', 'sum'),
                        Pedestrian_Injured = ('NUMBER OF PEDESTRIANS INJURED', 'sum'),
                        Pedestrian_Killed = ('NUMBER OF PEDESTRIANS KILLED', 'sum'),
                        Cyclist_Injured = ('NUMBER OF CYCLIST INJURED', 'sum'),
                        Cyclist_killed = ('NUMBER OF CYCLIST KILLED', 'sum'),
                        Motorist_Injured = ('NUMBER OF MOTORIST INJURED', 'sum'),    
                        Motorist_Killed = ('NUMBER OF MOTORIST KILLED', 'sum'),
                        AVG_Neighborhood_Population = ('POPULATION', 'mean'),
                        AVG_Neighborhood_Density = ('DENSITY', 'mean')
                          )

In [ ]:
df3=df3.reset_index() # Must add this statement to create an index after aggregation

In [ ]:
df3

I went deeper by aggregating by Borough, Neighborhood and Zip Code.

In [ ]:
df3a = df2.groupby(['BOROUGH','NEIGHBORHOOD','ZIP CODE'])\
                      .agg(
                        Incidents = ('COLLISION_ID', 'count'),
                        Neighborhoods = ('NEIGHBORHOOD', 'nunique'),
                        Injured = ('NUMBER OF PERSONS INJURED', 'sum'),
                        Killed = ('NUMBER OF PERSONS KILLED', 'sum'),
                        Pedestrian_Injured = ('NUMBER OF PEDESTRIANS INJURED', 'sum'),
                        Pedestrian_Killed = ('NUMBER OF PEDESTRIANS KILLED', 'sum'),
                        Cyclist_Injured = ('NUMBER OF CYCLIST INJURED', 'sum'),
                        Cyclist_killed = ('NUMBER OF CYCLIST KILLED', 'sum'),
                        Motorist_Injured = ('NUMBER OF MOTORIST INJURED', 'sum'),    
                        Motorist_Killed = ('NUMBER OF MOTORIST KILLED', 'sum'),
                        AVG_Neighborhood_Population = ('POPULATION', 'mean'),
                        AVG_Neighborhood_Density = ('DENSITY', 'mean')
                          )

In [ ]:
df3a=df3a.reset_index() # Must add this statement to create an index after aggregation

In [ ]:
df3a.sort_values(by='AVG_Neighborhood_Density', ascending=False)

I wanted to see if there was any correlation in the neighborhood's density and people getting injured. I created two maps to see side by side.

In [ ]:
df3b = df3a[['ZIP CODE', 'NEIGHBORHOOD','BOROUGH','Injured','Killed','Pedestrian_Injured','Pedestrian_Killed','Cyclist_Injured','Cyclist_killed','Motorist_Injured','Motorist_Killed']]

I searched online and found a visualization via Plotly that maps NYC zip code to the geo coordinates. 

https://github.com/OpenDataDE/State-zip-code-GeoJSON/blob/master/ny_new_york_zip_codes_geo.min.json

Then, I used the total injuries as the scale in my heat map. 

In [ ]:


from urllib.request import urlopen
import json
with urlopen('https://raw.githubusercontent.com/OpenDataDE/State-zip-code-GeoJSON/master/ny_new_york_zip_codes_geo.min.json') as response:
    counties = json.load(response)
    
    fig = px.choropleth(df3b, 
                    geojson=counties, 
                    locations='ZIP CODE', 
                    color='Injured',
                    color_continuous_scale="Viridis",
                    range_color=(1,750),
                    featureidkey="properties.ZCTA5CE10",
                    #scope="usa",
                    labels={'Injured':'Injured'}
                          )
fig.update_geos(fitbounds="locations", visible=False)
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})

I replicated the chart, but this time, used the Average Neighborhood desity as the measure. The chart did not support the hypothesis that injuries were more common in densely populated neighbordhoods.

In [ ]:
from urllib.request import urlopen
import json
with urlopen('https://raw.githubusercontent.com/OpenDataDE/State-zip-code-GeoJSON/master/ny_new_york_zip_codes_geo.min.json') as response:
    counties = json.load(response)
    
    fig = px.choropleth(df3a, 
                    geojson=counties, 
                    locations='ZIP CODE', 
                    color='AVG_Neighborhood_Density',
                    color_continuous_scale="Viridis",
                    range_color=(1,150000),
                    featureidkey="properties.ZCTA5CE10",
                    #scope="usa",
                    labels={'Density':'Density'}
                          )
fig.update_geos(fitbounds="locations", visible=False)
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})

There was virtually no correlation between the density and the injuries.

In [ ]:
corr = df3a['Injured'].corr(df3a['AVG_Neighborhood_Density'])

In [ ]:
corr

In [ ]:
print ("Correlation between ", 'Injured', " and ", 'AVG_Neighborhood_Density', "is: ", round(corr, 2))

In [ ]:
corr1 = df3a['Killed'].corr(df3a['AVG_Neighborhood_Density'])

In [ ]:
corr1

In [ ]:
print ("Correlation between ", 'Killed', " and ", 'AVG_Neighborhood_Density', "is: ", round(corr1, 2))

Appendix:

In [ ]:
df4 = df2.groupby(['BOROUGH','NEIGHBORHOOD'])\
                      .agg(
                        Incidents = ('COLLISION_ID', 'count'),
                        Neighborhoods = ('NEIGHBORHOOD', 'nunique'),
                        Injured = ('NUMBER OF PERSONS INJURED', 'sum'),
                        Killed = ('NUMBER OF PERSONS KILLED', 'sum'),
                        Pedestrian_Injured = ('NUMBER OF PEDESTRIANS INJURED', 'sum'),
                        Pedestrian_Killed = ('NUMBER OF PEDESTRIANS KILLED', 'sum'),
                        Cyclist_Injured = ('NUMBER OF CYCLIST INJURED', 'sum'),
                        Cyclist_killed = ('NUMBER OF CYCLIST KILLED', 'sum'),
                        Motorist_Injured = ('NUMBER OF MOTORIST INJURED', 'sum'),    
                        Motorist_Killed = ('NUMBER OF MOTORIST KILLED', 'sum'),    
                        AVG_Vehicles_Involved = ('TOTAL VEHICLES INVOLVED', 'mean'),    
                        AVG_Contributing_Factors = ('TOTAL CONTRIBUTING FACTORS', 'mean'),
                        AVG_Neighborhood_Population = ('POPULATION', 'mean'),
                        AVG_Neighborhood_Density = ('DENSITY', 'mean')
                          )

In [ ]:
df4=df4.reset_index() # Must add this statement to create an index after aggregation

In [ ]:
df4.head()

In [ ]:
df2[df['CONTRIBUTING FACTOR VEHICLE 1'] == 'Driver Inattention/Distraction']

In [ ]:
df5 = df2.groupby(['YEAR', 'BOROUGH','NEIGHBORHOOD'])\
                      .agg(
                        Incidents = ('COLLISION_ID', 'count'),
                        Days = ('CRASH DATE', 'nunique'),
                        Injured = ('NUMBER OF PERSONS INJURED', 'sum'),
                        Killed = ('NUMBER OF PERSONS KILLED', 'sum'),
                        Pedestrian_Injured = ('NUMBER OF PEDESTRIANS INJURED', 'sum'),
                        Pedestrian_Killed = ('NUMBER OF PEDESTRIANS KILLED', 'sum'),
                        Cyclist_Injured = ('NUMBER OF CYCLIST INJURED', 'sum'),
                        Cyclist_killed = ('NUMBER OF CYCLIST KILLED', 'sum'),
                        Motorist_Injured = ('NUMBER OF MOTORIST INJURED', 'sum'),    
                        Motorist_Killed = ('NUMBER OF MOTORIST KILLED', 'sum'),    
                        AVG_Vehicles_Involved = ('TOTAL VEHICLES INVOLVED', 'mean'),    
                        AVG_Contributing_Factors = ('TOTAL CONTRIBUTING FACTORS', 'mean'),
                        Population = ('POPULATION', 'mean'),
                        Density = ('DENSITY', 'mean')
                          )

In [ ]:
df5=df5.reset_index() # Must add this statement to create an index after aggregation

In [ ]:
df5.head()

In [ ]:
df6 = df2.groupby(['YEAR', 'BOROUGH','NEIGHBORHOOD','CONTRIBUTING FACTOR VEHICLE 1'])\
                      .agg(
                        Incidents = ('COLLISION_ID', 'count'),
                        Days = ('CRASH DATE', 'nunique'),
                        Injured = ('NUMBER OF PERSONS INJURED', 'sum'),
                        Killed = ('NUMBER OF PERSONS KILLED', 'sum'),
                        Pedestrian_Injured = ('NUMBER OF PEDESTRIANS INJURED', 'sum'),
                        Pedestrian_Killed = ('NUMBER OF PEDESTRIANS KILLED', 'sum'),
                        Cyclist_Injured = ('NUMBER OF CYCLIST INJURED', 'sum'),
                        Cyclist_killed = ('NUMBER OF CYCLIST KILLED', 'sum'),
                        Motorist_Injured = ('NUMBER OF MOTORIST INJURED', 'sum'),    
                        Motorist_Killed = ('NUMBER OF MOTORIST KILLED', 'sum'),    
                        AVG_Vehicles_Involved = ('TOTAL VEHICLES INVOLVED', 'mean'),    
                        AVG_Contributing_Factors = ('TOTAL CONTRIBUTING FACTORS', 'mean'),
                        Population = ('POPULATION', 'mean'),
                        Density = ('DENSITY', 'mean')
                          )

In [ ]:
df6=df6.reset_index() # Must add this statement to create an index after aggregation

In [ ]:
df6.head()

In [ ]:
df.groupby(['contributing_factor_vehicle_1','borough'])['crash_date'].count()

In [ ]:
df.groupby(['CONTRIBUTING FACTOR VEHICLE 1','BOROUGH'])['NUMBER OF PERSONS KILLED'].count()

In [ ]:
df.value_counts(['VEHICLE TYPE CODE 1', 'VEHICLE TYPE CODE 2', 'VEHICLE TYPE CODE 3','VEHICLE TYPE CODE 4', 'VEHICLE TYPE CODE 5'])